# Group Assignment 3 - UP221 Winter 2024
## Group Name: Food Access in LA County
### Madi Hamilton, Jessica Fay, Meaghan Woody, Branden Bohrnsen

Research Question - are there geographic disparities trends in food insecurity and coronary heart disease in Los Angeles County? 

Updates: new datasets (USC data merged with SPAs & grocery store walkability, regraphed the data for the poverty and predictor/outcome variables with new dataset, included function and loop for fast food data

In [1]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
import contextily as cx
import plotly.express as px
import osmnx as ox
import plotly.io as pio
import contextily as ctx
import networkx as nx
from matplotlib.patches import Patch
from matplotlib.lines import Line2D

In [4]:
# Import new, cleaned dataset for Grocery store walkability
grocery = pd.read_csv('Data/cleanedgrocery.csv')

In [5]:
grocery.head()

,Unnamed: 0,Record ID,Store Name,Store Type,Street Number,Street Name,Additional Address,City,State,Zip Code,Zip4,County,Latitude,Longitude,Authorization Date,End Date,end_date,open_date,latlon
0,63347,1057607,168 Market 803,Super Store,933,E Las Tunas Dr,,San Gabriel,CA,91776,1640,LOS ANGELES,34.10347,-118.0877,06/01/2012,,,2012,"[34.10347, -118.0877]"
1,63348,1171270,168 Market 806,Supermarket,17120,Colima Rd,,Hacienda Heights,CA,91745,6768,LOS ANGELES,33.99066,-117.9333,11/20/2014,,,2014,"[33.99066, -117.9333]"
2,63349,1192904,168 Market 805,Super Store,19725,Colima Rd,,Rowland Heights,CA,91748,3299,LOS ANGELES,33.98619,-117.8717,04/09/2015,,,2015,"[33.98619, -117.8717]"
3,63350,1263449,168 Market 5801,Super Store,1421,E Valley Blvd,,Alhambra,CA,91801,5239,LOS ANGELES,34.07928,-118.1099,10/24/2016,,,2016,"[34.07928, -118.1099]"
4,63415,1246643,365 by Whole Foods Market 10658,Super Store,2520,Glendale Blvd,,Los Angeles,CA,90039,3220,LOS ANGELES,34.10309,-118.2592,05/11/2016,,,2016,"[34.10309, -118.2592]"


# Part 4 - Grocery Store Walkability Maps

In [6]:
grocery['latlon']=grocery[['Latitude','Longitude']].values.tolist()

latlon = list(grocery['latlon'])
latlon

[[34.10347, -118.0877],
 [33.99066, -117.9333],
 [33.98619, -117.8717],
 [34.07928, -118.1099],
 [34.10309, -118.2592],
 [33.9968, -118.2652],
 [34.03999, -118.4647],
 [34.04008, -118.3476],
 [33.99646, -117.8884],
 [33.87275, -118.2966],
 [34.07961, -118.1015],
 [34.12338, -118.0597],
 [34.06256, -118.1334],
 [34.18811, -118.4661],
 [33.99284, -117.9318],
 [33.87005, -118.0824],
 [34.09719, -118.1227],
 [34.09811, -117.8902],
 [34.13054, -118.2524],
 [34.0587, -118.2917],
 [34.01667, -118.4109],
 [34.36922, -118.5121],
 [34.06255, -118.1013],
 [34.22293, -118.4487],
 [34.15782, -118.1433],
 [34.08597, -118.1792],
 [34.02152, -118.1662],
 [34.13769, -118.2418],
 [34.03273, -118.2926],
 [34.0325, -118.3622],
 [34.05085, -118.2169],
 [34.10663, -117.8058],
 [34.07098, -117.9211],
 [33.98114, -118.3584],
 [33.88205, -118.1239],
 [34.09085, -118.3019],
 [33.88786, -118.0647],
 [34.05133, -118.0469],
 [34.06411, -118.2389],
 [34.07156, -118.2917],
 [34.17622, -118.298],
 [34.14397, -118.699

In [ ]:
def isomap(latlon=latlon):
    network_type = 'walk'
    trip_times = [5,10,15] 
    meters_per_minute = 75 # travel distance per minute
    cmap = 'plasma'
    title = 'Grocery Store Walking Access'
    
    # download the street network
    G = ox.graph_from_point(latlon, network_type=network_type, dist = 2000)
    
    # project our network data to Web Mercator (measurements are in meters)
    G = ox.project_graph(G, to_crs='epsg:3857')
    
    # convert nodes and edges to geodataframes
    gdf_nodes, gdf_edges = ox.graph_to_gdfs(G)
    
    # get the bounding box coordinates
    minx, miny, maxx, maxy = gdf_nodes.geometry.total_bounds
    
    # calculate the centroid
    centroid_x = (maxx-minx)/2 + minx
    centroid_y = (maxy-miny)/2 + miny
    
    # use osmnx's distance.nearest_nodes command to get the id for the nearest node
    center_node = ox.distance.nearest_nodes(G,Y=centroid_y,X=centroid_x)
    
    # create a new column, calculate the time it takes to travel that edge
    gdf_edges['walk_time'] = gdf_edges['length']/meters_per_minute
    
    # assign a color hex code for each trip time isochrone
    iso_colors = ox.plot.get_colors(n=len(trip_times), 
                                    cmap=cmap, 
                                    start=0, 
                                    return_hex=True)
    
    # create a list of "zipped" time/colors
    time_color = list(zip(trip_times, iso_colors))
    
    # reverse the order so that outside nodes get associated first
    time_color.reverse()
    
    # loop through each trip time and associated color
    for time, color in list(time_color):
    
        # for each trip time, create an egograph of nodes that fall within that distance
        subgraph = nx.ego_graph(G, center_node, radius=time)
    
        # for each of those nodes, update the gdf_nodes dataframe and assign it with its associated distance color
        for node in subgraph.nodes():
            gdf_nodes.loc[node,'time'] = time
            gdf_nodes.loc[node,'color'] = color
    
    # the NaN values then need to be populated with a valid color
    gdf_nodes['color'].fillna('#cccccc', inplace=True)
    
    # dissolve the nodes by time
    # adding the "time" argument creates a separate geometry (multipoint in this case) for each unique time category
    isochrones = gdf_nodes.dissolve(by = "time")
    
    # for each row, create a convex hull
    isochrones = isochrones.convex_hull.reset_index(name='geometry')
    
    # reverse the order so that outer polygon gets drawn first
    isochrones.sort_values(by='time', ascending=False,inplace=True)
    
    # reverse the colors too!
    iso_colors.reverse()
    
    ########################
    
    #    Make the map!
    
    ########################
    # set up the subplots
    fig, ax = plt.subplots(figsize=(5,8))
    
    # add the isochrone boundary
    isochrones.boundary.plot(
        ax=ax,
        alpha=1,
        linestyle='--',
        color=iso_colors,
        lw=2
        ) 
    
    # add the isochrones
    isochrones.plot(
        ax=ax, 
        alpha=0.2, 
        categorical=True, # even though it is numeric, treat it as categorical
        color=iso_colors,
        )
    
    # add the center node in red
    gdf_nodes.loc[[center_node]].plot(
        ax=ax,
        color='r',
        marker='x',
        markersize=50
        )
    
    # build custom legend
    legend_elements = [
        # add the center node to the legend
        Line2D([0], [0], marker='x', color='red', linestyle='',label='Walkshed from this location', markersize=6),
        ]
    
    # sort back to small time to large time
    # time_color.sort(reverse=False)
    time_color.reverse()
    
    # loop through the list of time/colors and add each to the legend
    for time,color in list(time_color):
        legend_item = Patch(facecolor=color, edgecolor=color, linestyle='--',linewidth=1,label=str(time)+' minutes',alpha=0.4)
        legend_elements.append(legend_item)
    
    # add the legend
    ax.legend(handles=legend_elements,loc='lower left') # location options: upper/center/lower and left/center/right
    
    # add a title
    ax.set_title(title,fontsize=15,pad=10)
    
    # hide the axis
    ax.axis('off')
    
    # add the basemap
    ctx.add_basemap(ax,source=ctx.providers.CartoDB.Positron)

# list of neighborhoods
latlong = list(grocery['latlon'])

# call the function with a loop
for store in latlong:
    isomap(latlon=store)